# Queries in PyTables

> Objectives:
> * Query HDF5 files without loading them in-memory
> * How to query normalized and denormalized tables
> * Index columns in tables for accelerating queries

In [1]:
import os
import numpy as np
import pandas as pd
import tables

In [2]:
%ls -lh structuring compression

compression:
total 121712
-rw-r--r--  1 faltet  staff   5.0M May 20 13:07 blosc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.0M May 20 13:07 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 20 13:07 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.8M May 20 13:07 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staff   5.5M May 20 13:07 blosc-snappy-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 20 13:07 blosc-zlib-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.3M May 20 13:07 blosc-zstd-5-shuffle.h5
-rw-r--r--  1 faltet  staff   4.1M May 20 13:07 bzip2-5-shuffle.h5
-rw-r--r--  1 faltet  staff    17M May 20 13:07 no-compressed.h5
-rw-r--r--  1 faltet  staff   4.2M May 20 13:07 zlib-5-shuffle.h5

structuring:
total 438952
-rw-r--r--  1 faltet  staff   7.2M May 20 13:10 blosc-blosclz-5-shuffle.h5
-rw-r--r--  1 faltet  staff   7.8M May 20 13:10 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 faltet  staff   6.6M May 20 13:10 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 faltet  staf

## Querying in PyTables

### Denormalized tables

In [3]:
h5denorm = "structuring/blosc-zstd-5-shuffle.h5"
h5file = tables.open_file(h5denorm)
h5lens = h5file.root.lens

In [4]:
h5lens

/lens (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "rating": Int8Col(shape=(), dflt=0, pos=1),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=2),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=3),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (402,)

In [5]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 946 ms, sys: 147 ms, total: 1.09 s
Wall time: 1.01 s


In [6]:
ratings

[0, 4, 15, 28, 18, 3]

In [7]:
h5file.close()

Querying denormalized tables is easy as pie.  Let's see how to manage normalized ones.

### Normalized tables

In [8]:
h5norm = "compression/blosc-zstd-5-shuffle.h5"
h5file = tables.open_file(h5norm)
h5ratings = h5file.root.ratings
h5movies = h5file.root.movies

In [9]:
h5ratings

/ratings (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)

In [10]:
h5movies

/movies (Table(3883,), shuffle, blosc:zstd(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt=b'', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt=b'', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)

In [11]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 319 ms, sys: 41.9 ms, total: 361 ms
Wall time: 381 ms


In [12]:
ratings

[0, 4, 15, 28, 18, 3]

In [13]:
h5file.close()

So, the query in the normalized version is more than 2~3x faster than using the denormalized file.  However, this is just a simple example, and in general experimentation should be done so as to determine the best layout for your data.

## Indexing

Indexing is a general technique for adding data structures that can accelerate queries.  Let's see how PyTables makes use of this.

### Denormalized case

In [14]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-denorm-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5denorm, h5idx)

'movielens-denorm-indexed.h5'

In [15]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")

In [16]:
# Create an index for the 'title' column
h5lens = h5i.root.lens
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5lens.cols.title.create_csindex(filters=blosc_filter)

CPU times: user 1.45 s, sys: 265 ms, total: 1.72 s
Wall time: 1.79 s


1000209

In [17]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 9.26 ms, sys: 1.77 ms, total: 11 ms
Wall time: 12.2 ms


Ok, so this time is 100x less than without using indexing.  What if we index the `rating` column too?

In [18]:
ratings

[0, 4, 15, 28, 18, 3]

In [19]:
# Create an index for the rating column
%time h5lens.cols.rating.create_csindex(filters=blosc_filter)

CPU times: user 402 ms, sys: 34.9 ms, total: 437 ms
Wall time: 497 ms


1000209

In [20]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 3.09 ms, sys: 106 µs, total: 3.2 ms
Wall time: 3.47 ms


Ok, so although small, this represents another improvement in performance.

In [21]:
ratings

[0, 4, 15, 28, 18, 3]

In [22]:
h5i.close()

### Normalized case

In [23]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-norm-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5norm, h5idx)

'movielens-norm-indexed.h5'

In [24]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [25]:
# Create an index for the rating column
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5ratings.cols.rating.create_csindex(filters=blosc_filter)

CPU times: user 341 ms, sys: 32.8 ms, total: 374 ms
Wall time: 437 ms


1000209

In [26]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 297 ms, sys: 36.2 ms, total: 333 ms
Wall time: 312 ms


Hmm, in this case indexing the rating column has not served to accelerate the query (at first sight at least).

In [27]:
ratings

[0, 4, 15, 28, 18, 3]

In [28]:
# Create an index for the movie_id column
%time h5ratings.cols.movie_id.create_csindex(filters=blosc_filter)

CPU times: user 312 ms, sys: 29.6 ms, total: 342 ms
Wall time: 389 ms


1000209

In [29]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 30.4 ms, sys: 2.75 ms, total: 33.2 ms
Wall time: 49.5 ms


This time we see a better acceleration in the query, but cannot compete with the query speed for the denormalized case (which is ~10x faster).

In [30]:
ratings

[0, 4, 15, 28, 18, 3]

In [31]:
h5i.close()

In [32]:
%ls -lh movielens*

-rw-r--r--  1 faltet  staff   9.9M May 20 13:13 movielens-denorm-indexed.h5
-rw-r--r--  1 faltet  staff   9.6M May 20 13:13 movielens-norm-indexed.h5

movielens-1m:
total 11472
-rw-r--r--  1 faltet  staff   5.1K May 19 10:08 README
-rw-r--r--  1 faltet  staff    56K May 19 10:08 movies.dat.gz
-rw-r--r--  1 faltet  staff   5.5M May 19 10:08 ratings.dat.gz
-rw-r--r--  1 faltet  staff    39K May 19 10:08 users.dat.gz


## Exercise

We have not created an index for the title for the normalized case.  Create such an index and determine if there is a noticeable speed-up or not.  Explain why you think that is the case.  Note: the times for a cold query can be **significatively** different from a hot query.

In [33]:
## Copy the original PyTables table into another file
import shutil
h5idx2 = "movielens-norm-indexed2.h5"
if os.path.exists(h5idx2):
    os.unlink(h5idx2)
shutil.copyfile(h5idx, h5idx2)

'movielens-norm-indexed2.h5'

In [34]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx2, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [35]:
# Create an index for the movie_id column
%time h5movies.cols.title.create_csindex(filters=blosc_filter)

CPU times: user 17.4 ms, sys: 1.83 ms, total: 19.3 ms
Wall time: 37.5 ms


3883

In [36]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 197 ms, sys: 26 ms, total: 223 ms
Wall time: 226 ms


In [37]:
ratings

[0, 4, 15, 28, 18, 3]

In [38]:
h5i.close()

So the first time that the query is done after the cache is built (cold query), the time has been reduced a bit but not too much.  For subsequent queries (hot queries), the times are better, but not reaching the denormalized table either.